In [56]:
import pandas as pd
import numpy as np
#sklearn doesn't automatically import subpackages but for formality I've included it in the module imports declaration section
import sklearn 
from sklearn import preprocessing
import opendatasets 
import matplotlib.pyplot as plt
import datetime


In [3]:
#Downloads the dataset to the current folder
import opendatasets 
ccfd_dataset_url = "https://www.kaggle.com/datasets/kartik2112/fraud-detection?select=fraudTrain.csv"
opendatasets.download(ccfd_dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading fraud-detection.zip to .\fraud-detection


100%|██████████| 202M/202M [05:45<00:00, 613kB/s] 


In [7]:
transactions_train = pd.read_csv("./fraud-detection/fraudTrain.csv")
transactions_test = pd.read_csv("./fraud-detection/fraudTest.csv")
transactions_train.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,6.483370e+05,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03
std,3.743180e+05,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00
25%,3.241685e+05,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00
50%,6.483370e+05,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00
75%,9.725055e+05,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [51]:
#Performs feature selection, engineering and extraction
def process_features(df: pd.DataFrame)->pd.DataFrame:
    new_df = df.drop(columns=["Unnamed: 0", "cc_num", "city", "street", "state", "zip", "trans_num"], axis=1)
    #Generate age and drop dob
    current_year = datetime.datetime.now().year
    new_df["age"]= new_df["dob"].apply(lambda x: current_year - int(x.split("-")[0]))
    new_df = new_df.drop(columns=["dob"], axis=1)
    #Remove the weird fraud_ thing that comes before a merchants name and rename merchant to merch_name
    new_df["merchant"] = new_df["merchant"].apply(lambda x: x.split("fraud_")[1]) 
    new_df = new_df.rename(columns={"merchant":"merch_name"})
    #Combine first and last name into full name then drop the them
    new_df["full_name"] = new_df["first"]+" "+new_df["last"] 
    new_df = new_df.drop(columns=["first", "last"], axis=1)
    #Drop unix time
    new_df = new_df.drop(columns=["unix_time"], axis=1)
    datetime_format = "%Y-%m-%d %H:%M:%S"
    new_df["dow"] = new_df["trans_date_trans_time"].apply(lambda x: datetime.datetime.strptime(x, datetime_format).weekday())
    new_df["dom"] = new_df["trans_date_trans_time"].apply(lambda x: int(x.split("-")[2].split(" ")[0]))
    new_df["month"] = new_df["trans_date_trans_time"].apply(lambda x: int(x.split("-")[1]))
    new_df["year"] = new_df["trans_date_trans_time"].apply(lambda x: int(x.split("-")[0]))
    def process_time(time:str)->float:
        times = time.split(" ")[1].split(":")
        return float(times[0])+float(times[1])/60
        #return float(times[0])+float(times[1])/60+float(times[2])/3600
    new_df["time"] = new_df["trans_date_trans_time"].apply(process_time)
    new_df.drop(columns=["trans_date_trans_time"], axis=1)
    new_column_order = ["full_name", "gender", "age", "job", "lat", "long", "city_pop", "merch_name", "merch_lat", "merch_long", "amt", "category", "time", "dow", "dom", "month", "year", "is_fraud"]
    return new_df.reindex(columns=new_column_order)
    


transactions_processed_features = process_features(transactions_train)



In [61]:
def label_encode(df: pd.DataFrame)->pd.DataFrame:
    cat_features = ["full_name", "gender", "job", "merch_name", "category"]
    encoder = preprocessing.LabelEncoder()
    df_copy = df.copy()
    for feat in cat_features:
        df_copy[feat] = encoder.fit_transform(df_copy[feat])
    return df_copy


transactions_encoded = label_encode(transactions_processed_features)
transactions_encoded
corr_matrix = transactions_encoded.corr()
corr_matrix["is_fraud"].sort_values(ascending=False)

is_fraud      1.000000
amt           0.219404
category      0.020205
time          0.013831
age           0.012378
gender        0.007642
dom           0.003848
year          0.003004
city_pop      0.002136
lat           0.001894
merch_lat     0.001741
dow           0.001739
merch_long    0.001721
long          0.001721
job          -0.000093
merch_name   -0.000536
full_name    -0.003285
month        -0.012409
Name: is_fraud, dtype: float64